In [20]:
import os
import sys
import requests
import json
import operator
import time
import datetime
if not os.path.exists('.\\output'):
    os.makedirs('.\\output')
    
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
time.sleep(1)
print('_______________________________________________')
print("Python Version: "+sys.version)
print("Requests Version: "+requests.__version__)
print("Json Version: "+json.__version__)

2018-10-11 11:57:37
_______________________________________________
Python Version: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Requests Version: 2.19.1
Json Version: 2.0.9


In [21]:
def writeText(text, path, mode = 'a'):
    with open (path, mode) as textout:
        textout.write((text))
        
def writeJson(json, path, mode = 'w'):
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
def writeCsv(listOut, outputFile):
    import csv
    with open (outputFile, "w", newline='', encoding = 'utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)

In [22]:
def getToken():
    return open('../token/token.txt', 'r').read()

In [23]:
def getJsonDocs(page = 1, itemsperpage = 200):
    jsondata ='dummy'
    list_return = []

    while jsondata:
        AUTH_TOKEN = getToken()
        try:
            r = requests.get("https://app.23degrees.io/services/pub/api/v1/opendata/getWorldBankDataSetsPaginated/"+str(page)+"/"+str(itemsperpage)
                         ,timeout=None
                         ,headers={  'Content-Type': 'application/json'
                                    , 'Authorization' : 'Bearer '+ getToken()
                                   }
                        )    
            jsondata = r.json()
            for elem in jsondata:
                list_return.append(elem)
        except requests.exceptions.HTTPError as err:
            print(err)
        page += 1
    return list_return

In [24]:
list_js_docs = getJsonDocs(1,200)

In [25]:
list_js_tags = []
list_js_notags = []
list_unique_tags = []
dict_tags = {}
for i, jsondoc in enumerate(list_js_docs):
    jstmp = json.dumps(jsondoc, sort_keys=True,indent=4, separators=(',', ': '))
    if 'tags' in jstmp:
        list_js_tags.append(jstmp)
        for tag in jsondoc['typeSpecific']['context']['tags']:
            if tag not in list_unique_tags:
                list_unique_tags.append(tag)
                dict_tags[tag] = 1
            else:
                dict_tags[tag] += 1
                
    else:
        list_js_notags.append(jstmp)

list_tags_sorted = list(sorted(dict_tags.items(), key=lambda x: x[1], reverse = True))

In [26]:
print('Json-Files: ', len(list_js_docs))
print('Json-Files with tags: ', len(list_js_tags))
print('Json-Files without tags: ', len(list_js_notags))
print('Unique tags: ', len(list_tags_sorted))

Json-Files:  4580
Json-Files with tags:  3573
Json-Files without tags:  1007
Unique tags:  2848
